
See original StyleGAN2 ADA github repo here: https://github.com/NVlabs/stylegan2-ada-pytorch


In [ ]:
!pip install pyspng ninja imageio-ffmpeg==0.4.3


In [ ]:
!pip install gdown 

In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch
%cd ./stylegan2-ada-pytorch

### Preprocessing Dataset

In [ ]:
import os
from PIL import Image
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# Source (read-only)
src_dir = "/kaggle/input/fashion-dataset/train_images"

# Destination (writable)
dst_dir = "/kaggle/working/train_images"
os.makedirs(dst_dir, exist_ok=True)

# Lấy danh sách ảnh
files = [f for f in os.listdir(src_dir) if f.lower().endswith((".png", ".jpg", ".jpeg"))]

def process(file):
    try:
        src_path = os.path.join(src_dir, file)
        dst_path = os.path.join(dst_dir, file)
        img = Image.open(src_path).convert("RGB")
        img = img.resize((256, 256), Image.LANCZOS)
        img.save(dst_path)
    except Exception as e:
        print(f"Error processing {file}: {e}")

# Dùng ThreadPoolExecutor để resize đa luồng + tqdm để hiển thị tiến trình
with ThreadPoolExecutor(max_workers=8) as executor:
    list(tqdm(executor.map(process, files), total=len(files)))

print("Done! All images are 256x256 RGB in", dst_dir)


In [ ]:
!python --version


In [ ]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device:", torch.cuda.get_device_name(0))

In [ ]:
!python dataset_tool.py --source=/kaggle/working/train_images --dest=./datasets/fashion_image.zip

In [ ]:
!python train.py --help

### Training will take a long time, make sure you have enough GPU hours available (minimum 10 hours required for good results)
Do !python train.py --help to see the different optional arguments available.

Resume from your previous checkpoint of notebook times out. Get output model from: ./stylegan2-ada-pytorch/results

In [ ]:
!ls /kaggle/input/stylegan2-3200kimg/pytorch/default/1/network-snapshot-003200.pkl

In [ ]:
!python train.py --resume=/kaggle/input/stylegan2-3200kimg/pytorch/default/1/network-snapshot-003200.pkl --gpus=2 --batch=16 --outdir=/kaggle/working/result_from3200kimg --snap=100 --cfg=paper256 --data=./datasets/fashion_image.zip --aug=noaug --mirror=True --metrics=None  --kimg=800

Stopping training as example notebook